<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Extract Transform Load (ETL) Lab**


Estimated time needed: **30** minutes


## Objectives

After completing this lab you will be able to:

-   Read CSV and JSON file types.
-   Extract data from the above file types.
-   Transform data.
-   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [38]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
from pandas import DataFrame as pddf     # importing Pandas DataFrame attribute separately to try fix an error
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [39]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2021-07-03 15:58:36--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: 'source.zip'

     0K ..                                                    100%  416M=0s

2021-07-03 15:58:37 (416 MB/s) - 'source.zip' saved [2707/2707]



## Unzip Files


In [40]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


## Set Paths


In [41]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [51]:
# def extract_from_csv(file_to_process):
#     dataframe = pd.read_csv(file_to_process)
#     return dataframe

def csvExtract(csvFile):
    dframe = pd.read_csv(csvFile)
    return dframe

### JSON Extract Function


In [52]:
# def extract_from_json(file_to_process):
#     dataframe = pd.read_json(file_to_process,lines=True)
#     return dataframe

def jsonExtract(jsonFile):
    dframe = pd.read_json(jsonFile,lines=True)
    return dframe

### XML Extract Function


In [74]:
# def extract_from_xml(file_to_process):
#     dataframe = pd.DataFrame(columns=["name", "height", "weight"])
#     tree = ET.parse(file_to_process)
#     root = tree.getroot()
#     for person in root:
#         name = person.find("name").text
#         height = float(person.find("height").text)
#         weight = float(person.find("weight").text)
#         dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
#     return dataframe

def xmlExtract(xmlFile):
    dframe = pddf(columns=["name", "height", "weight"])
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dframe = dframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dframe


### Extract Function


In [75]:
#  def extract():
#    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
#     
#     #process all csv files
#     for csvfile in glob.glob("*.csv"):
#         extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
#         
#     #process all json files
#     for jsonfile in glob.glob("*.json"):
#         extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
#     
#     #process all xml files
#     for xmlfile in glob.glob("*.xml"):
#         extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
#         
#     return extracted_data

def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # This creates the empty dataframe to append to
    
    # First, process all CSV files via loop
    for csvFile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(csvExtract(csvFile), ignore_index=True)
        
    # Second, process all json files
    for jsonFile in glob.glob("*.json"):
        extracted_data = extracted_data.append(jsonExtract(jsonFile), ignore_index=True)
        
    # Lastly, process all xml files
    for xmlFile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(xmlExtract(xmlFile), ignore_index=True)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [76]:
# def transform(data):
#         #Convert height which is in inches to millimeter
#         #Convert the datatype of the column into float
#         #data.height = data.height.astype(float)
#         #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
#         data['height'] = round(data.height * 0.0254,2)
#         
#         #Convert weight which is in pounds to kilograms
#         #Convert the datatype of the column into float
#         #data.weight = data.weight.astype(float)
#         #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
#         data['weight'] = round(data.weight * 0.45359237,2)
#         return data

def transform(data): # convert height from inches to metres, and weight from pounds to kilos
    data['height'] = round(data.height * 0.0254,2)
    data ['weight'] = round(data.weight * 0.45359237,2)
    return data

## Loading


In [77]:
# def load(targetfile,data_to_load):
#     data_to_load.to_csv(targetfile)  

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

## Logging


In [78]:
# def log(message):
#     timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
#     now = datetime.now() # get current timestamp
#     timestamp = now.strftime(timestamp_format)
#     with open("logfile.txt","a") as f:
#         f.write(timestamp + ',' + message + '\n')

def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("Logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [81]:
log("ETL Job Started")

In [82]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [ ]:
log("ETL Job Ended")

# Exercise


Using the example above complete the exercise below.


## Download Files


In [28]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2021-07-02 21:49:44--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: 'datasource.zip'

     0K ....                                                  100%  900M=0s

2021-07-02 21:49:45 (900 MB/s) - 'datasource.zip' saved [4249/4249]



## Unzip Files


In [29]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [30]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### Question 1: CSV Extract Function


In [31]:
# Add the CSV extract function below


def csvExtract(csvFile):
    dataframe = pd.read_csv(csvFile)
    return dataframe

<details><summary>Click here for the solution</summary>
    
```
    
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
```
</details>


### Question 2: JSON Extract Function


In [32]:
# Add the JSON extract function below

def jsonExtract(jsonFile):
    dataframe = pd.read_json(jsonFile, lines=True)
    return dataframe


<details><summary>Click here for the solution</summary>
    
```
    
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe
```
</details>


### Question 3: XML Extract Function


In [40]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.

def xmlExtract(xmlFile):
    dataframe = pd.dataframe(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    for vehicle in root:
        car_model = vehicle.find("car_model").text
        year_of_manufacture = float(vehicle.find("year_of_manufacture").text)
        price = float(vehicle.find("price").text)
        fuel = float(vehicle.find("fuel").text)
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe    


<details><summary>Click here for the solution</summary>
    
```
    
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe
```
</details>


### Question 4: Extract Function

Call the specific extract functions you created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [55]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvFile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(csvExtract(csvFile), ignore_index=True)
        
    #process all json files
    for jsonFile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(jsonExtract(jsonFile), ignore_index=True)
    
    #process all xml files
    for xmlFile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(xmlExtract(xmlFile), ignore_index=True)
        
    return extracted_data

<details><summary>Click here for the solution</summary>
    
```
    
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
```
</details>


## Transform


### Question 5: Transform

Round the `price` columns to 2 decimal places


In [56]:
# Add the transform function below

def transform(data):
    data['price'] = round(data.price,2)
    return data

<details><summary>Click here for the solution</summary>
    
```

def transform(data):
        data['price'] = round(data.price, 2)
        return data

```
</details>
```


## Loading


### Question 6: Load


In [57]:
# Add the load function below

def loadData(targetfile,loadFile):
    targetfile.to_csv(loadFile)


<details><summary>Click here for the solution</summary>
    
```

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

```
</details>
```


## Logging


### Question 7: Log

Make sure to change the name of the logfile to the one specified in the set paths section. Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [58]:
# Add the log function below

def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')


<details><summary>Click here for the solution</summary>
    
```

def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

```
</details>
```


## Running ETL Process


### Question 8: ETL Process

Run all functions to extract, transform, and load the data. Make sure to log all events using the `log` function. Place your code under each comment.


In [35]:
# Log that you have started the ETL process
log("ETL Process Started")

# Log that you have started the Extract step

log("Extract step started")

# Call the Extract function

extracted_data = extract()

# Log that you have completed the Extract step

log("Extract step complete")

# Log that you have started the Transform step

log("Transform step started")

# Call the Transform function

transformeddata = transform(extracted_data)

# Log that you have completed the Transform step

log("Transform step complete")

# Log that you have started the Load step

log("Load step started")

# Call the Load function

loadData(targetfile,loadFile)

# Log that you have completed the Load step

log("Load step complete")

# Log that you have completed the ETL process

log("ETL process complete")

AttributeError: module 'pandas' has no attribute 'dataframe'

<details><summary>Click here for the solution</summary>
    
```

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")

```
</details>
```


## Authors


Ramesh Sannareddy

Joseph Santarcangelo

Azim Hirjani


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
